In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_union

In [10]:
df_train = pd.read_csv('datasources/jigsaw1/train.csv').fillna(' ')
df_test = pd.read_csv('datasources/jigsaw1/train.csv').fillna(' ')

train_text = df_train['comment_text']
test_text = df_test['comment_text']
all_text = pd.concat([train_text, test_text])

all_text = all_text.sample(10)
all_text.shape

(10,)

In [4]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 1),
    max_features=30000)
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 4),
    max_features=30000)
vectorizer = make_union(word_vectorizer, char_vectorizer, n_jobs=2)

vectorizer.fit(all_text)
train_features = vectorizer.transform(train_text)
test_features = vectorizer.transform(test_text)

In [8]:
scores = []
categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
submission = pd.DataFrame.from_dict({'id': test['id']})
for cat in categories:
    train_target = train[cat]
    classifier = LogisticRegression(solver='sag')

    cv_score = np.mean(cross_val_score(
        classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(cat, cv_score))

    classifier.fit(train_features, train_target)
    submission[cat] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

submission.to_csv('submission.csv', index=False)

CV score for class toxic is 0.9472584047080428
CV score for class severe_toxic is 0.9832477971291397
CV score for class obscene is 0.9689285539200717
CV score for class threat is 0.9694842098360095
CV score for class insult is 0.9635113108894156
CV score for class identity_hate is 0.9642953130543558
Total CV score is 0.9661209315895057
